# Convolutional Neural Networks
Source: https://www.tensorflow.org/alpha/tutorials/images/intro_to_cnns

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

print(tf.__version__)

2.0.0-alpha0


## Download and Prepare MNIST data

In [4]:
(train_images, train_labels),(test_images, test_labels) = datasets.mnist.load_data()
train_images = train_images.reshape((60000,28,28,1))
test_images = test_images.reshape((10000,28,28,1))
#Normalize pixel values to be in between 0 and 1
train_images, test_images = train_images/255.0, test_images/255.0

## Create the Convolutional Base

In [6]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3),activation='relu', input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


##  Add Dense Layer on Top

In [8]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(10, activation = 'softmax'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

## Compile and Train Model

In [11]:
model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

model.fit(train_images,train_labels, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 92s 2ms/sample - loss: 0.1425 - accuracy: 0.9553
Epoch 2/5
60000/60000 [==============================] - 90s 2ms/sample - loss: 0.0463 - accuracy: 0.9859
Epoch 3/5
60000/60000 [==============================] - 88s 1ms/sample - loss: 0.0323 - accuracy: 0.9896
Epoch 4/5
60000/60000 [==============================] - 80s 1ms/sample - loss: 0.0248 - accuracy: 0.9925
Epoch 5/5
60000/60000 [==============================] - 81s 1ms/sample - loss: 0.0199 - accuracy: 0.9942


## Evaluate the Model

In [12]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("test_acc: ", test_acc)

10000/10000 [==============================] - 4s 355us/sample - loss: 0.0275 - accuracy: 0.9919
test_acc:  0.9919


## Create a Saved Model

In [13]:
import time
saved_model_path = "./saved_models/{}".format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

W0314 21:54:28.636028 21404 deprecation.py:323] From c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\saved_model\signature_def_utils_impl.py:253: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
W0314 21:54:28.651643 21404 tf_logging.py:161] Export includes no default signature!
W0314 21:54:32.354798 21404 tf_logging.py:161] Export includes no default signature!


'./saved_models/1552580662'

##  Reload a fresh keras model from the saved model

In [14]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [17]:
new_model.predict(test_images).shape

(10000, 10)

In [18]:
# The model has to be compiled before evaluating.
# This step is not required if the saved model is only being deployed.

new_model.compile(optimizer=model.optimizer,  # keep the optimizer that was loaded
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

10000/10000 [==============================] - 8s 755us/sample - loss: 0.0275 - accuracy: 0.9919
Restored model, accuracy: 99.19%
